# Orientation Mapping

Orientation Mapping in `pyxem` is very fast and powerful!  There are many tools for aligning your data like:

- Aligning your zero beam with [center_direct_beam](https://pyxem.readthedocs.io/en/stable/examples/processing/centering_the_zero_beam.html#sphx-glr-examples-processing-centering-the-zero-beam-py)
- Removing astigmatism with  [apply_affine_transformation](https://pyxem.readthedocs.io/en/stable/examples/processing/determining_ellipticity.html#sphx-glr-examples-processing-determining-ellipticity-py)
- Removing the background using [subtract_background](https://pyxem.readthedocs.io/en/stable/examples/processing/background_subtraction.html#sphx-glr-examples-processing-background-subtraction-py)
- Get Polar coordainates using [2D azimuthal integration](https://pyxem.readthedocs.io/en/stable/examples/processing/azimuthal_integration.html#sphx-glr-examples-processing-azimuthal-integration-py)

For more information see this paper:

```
Niels Cautaerts, Phillip Crout, Håkon W. Ånes, Eric Prestat, Jiwon Jeong, Gerhard Dehm, Christian H. Liebscher,
Free, flexible and fast: Orientation mapping using the multi-core and GPU-accelerated template matching capabilities in the Python-based open source 4D-STEM analysis toolbox Pyxem,
Ultramicroscopy,
Volume 237,
2022,
113517,
ISSN 0304-3991,
https://doi.org/10.1016/j.ultramic.2022.11351
```

<center><img src="Images/OrientationMapping.jpg" alt="Orient1" height="500" width="500"> <img src="Images/OrientationMapping2.jpg" alt="Orient1" height="600" width="500"></center>


---
**NOTE**

This uses pyxem==0.19.0 and diffsims==0.6.0.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
import zarr

In [ ]:
# load the data from a zip store zspy file
store = zarr.ZipStore("data/cuzipProcessed.zspy")
s = hs.load(store, lazy=True)

In [ ]:
# get the azimuthal integral 2d
az = s.get_azimuthal_integral2d(npt=250, mean=True,radial_range=[0,3.] )

In [ ]:
# transfer the navigator
az.navigator= s.navigator

In [ ]:
# plot the azimuthal Integral 

## Creating a Simulation Library

We can make a simple simulation library using `diffsims`. Here we sample in the (reduced) S2 space removing any orieations that are identical due to symetry.  Additionally, because we correlate in the polar direction we don't have to consider the first Euler angle!

In [ ]:
from orix.sampling import get_sample_reduced_fundamental
from orix.vector import Vector3d
from orix import plot
import matplotlib.pyplot as plt
from orix.crystal_map import Phase
from diffsims.generators.simulation_generator import SimulationGenerator

In [ ]:
# load the cif file
p = Phase.from_cif("data/Cu.cif")

In [ ]:
# get the reduced set of points
grid = get_sample_reduced_fundamental(1, point_group=p.point_group)

In [ ]:
# plotting the rotations in 2D on the sterographic projection
vectors = grid*Vector3d.zvector()
subplot_kw = {"projection": "ipf", "symmetry": p.point_group, "direction": Vector3d.zvector}
fig = plt.figure(figsize=(4, 4))

ax0 = fig.add_subplot(111, **subplot_kw)
ax0.scatter(vectors, alpha=0.05)
_ = ax0.set_title(f"CU Samping, Z")


In [ ]:
# Create a simulation
gen = SimulationGenerator(200, minimum_intensity=.05, shape_factor_model="linear")
sim = gen.calculate_diffraction2d(phase=p, rotation=grid, max_excitation_error=0.15, reciprocal_radius=3, with_direct_beam=False)

In [ ]:
# plot the simulation
sim.plot(interactive=True, show_labels=True,)

In [ ]:
#close the plot
plt.close("all")

In [ ]:
# get the best fit orientation. 
orient = az.get_orientation(sim,n_best=10, frac_keep=1, normalize_templates=True)

In [ ]:
# plot the phase markers (lazily :). This helps us to verify that we have the right parameters without computing everything!
m= orient.to_single_phase_markers(include_intensity=True, intesity_scale=10)
s.plot(vmax="99.9th")
s.add_marker(m)

In [ ]:
plt.close("all")

In [ ]:
orient.compute()

In [ ]:
nav = orient.to_navigator()

In [ ]:
nav.plot()

In [ ]:
%matplotlib ipympl
ipf_markers = orient.to_ipf_markers()
vector_markers = orient.to_single_phase_markers()
s.plot(navigator=nav, vmax="99.9th")
s.add_marker(ipf_markers)
s.add_marker(vector_markers)

In [ ]:
orient.isig[1].T.plot()